In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read in the data from a JSON file
df = pd.read_json('indeed-data-jobs-FINAL.json')
df.sample(5)

,job_title,company,location,is_remote,salary,post_date,date_accessed,job_description
778,"Machine Learning Engineer, Cameos",Snapchat,"Los Angeles, CA 90291",0,,14 days ago,2021-03-05,Snapchat is the camera used by millions of peo...
261,Data Analyst-Reporting,Integrity Marketing Group,"Dallas, TX",0,,2 days ago,2021-03-05,Company Description Company Description Integr...
800,Machine Learning Engineer,Pictor Labs,"Los Angeles, CA 90025",1,,13 days ago,2021-03-05,Spun off from UCLA Engineering and School of M...
790,Machine Learning Engineer - Computer Vision ( ...,"Cardinal Intellectual Property, Inc",Remote,1,,29 days ago,2021-03-05,***Remote Contract Opportunity***Machine Learn...
105,Data Scientist,"USG, Inc","Charlotte, NC",0,$60 - $65 an hour,30+ days ago,2021-03-05,"Title: Data ScientistLocation: Charlotte, NC (..."


In [3]:
# these were probably removed so I'll remove them from the data
df[df.job_description == '']

,job_title,company,location,is_remote,salary,post_date,date_accessed,job_description
303,,,,0,,,2021-03-05,
463,,,,0,,,2021-03-05,
586,,,,0,,,2021-03-05,
755,,,,0,,,2021-03-05,


In [4]:
# REMOVED ROWS ABOVE
df = df[df.job_description != ''].reset_index(drop=True)
df

,job_title,company,location,is_remote,salary,post_date,date_accessed,job_description
0,Data Entry,TransPerfect Translations Inc.,Remote,1,,6 days ago,2021-03-05,We are looking for Data Entry Clerks to join o...
1,Data Scientist,ForMotiv,Remote,1,"$75,000 - $120,000 a year",30+ days ago,2021-03-05,Has it ever occurred to you that as the Intern...
2,Data Scientist,Redzara.com,Remote,1,$35 - $80 an hour,10 days ago,2021-03-05,Only GC / EAD only. No C2CBackground screening...
3,Data Scientist,Nova Collective,Remote,1,$35 - $48 an hour,24 days ago,2021-03-05,Are you a data scientist who is really excited...
4,Early Career Data Scientist - Applied Math,Pacific Northwest National Laboratory,"Seattle, WA",0,,1 day ago,2021-03-05,Organization and Job ID Job ID: 311747 Directo...
...,...,...,...,...,...,...,...,...
893,"Machine Learning Engineer, Ads Relevance",Pinterest,"Palo Alto, CA",0,,24 days ago,2021-03-05,About Pinterest: Millions of people across th...
894,Machine Learning: AI Engineer,"ALTRON, Inc.","Fairfax, VA",0,,30+ days ago,2021-03-05,Altron Inc. is seeking a Machine Learning (ML)...
895,Machine Learning Engineer,Global Payments,"Alpharetta, GA 30022",0,,24 days ago,2021-03-05,"Every day, Global Payments makes it possible f..."
896,Applied Machine Learning Engineer,Anomaly,"New York, NY 10012",0,,30+ days ago,2021-03-05,About Anomaly Anomaly is a healthcare technol...


In [5]:
# rows, columns
df.shape

(898, 8)

In [6]:
# data summary
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 898 entries, 0 to 897
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job_title        898 non-null    object
 1   company          898 non-null    object
 2   location         898 non-null    object
 3   is_remote        898 non-null    int64 
 4   salary           898 non-null    object
 5   post_date        898 non-null    object
 6   date_accessed    898 non-null    object
 7   job_description  898 non-null    object
dtypes: int64(1), object(7)
memory usage: 56.2+ KB


# Creating Labels
### I'll need to create a script that uses the job title to create the label
I already see that data entry isn't what I'm looking for so I'll need to iterate over all entries to purge jobs like this that don't fall into 1 of my 4 categories

In [7]:
df.job_title

0                                      Data Entry
1                                  Data Scientist
2                                  Data Scientist
3                                  Data Scientist
4      Early Career Data Scientist - Applied Math
                          ...                    
893      Machine Learning Engineer, Ads Relevance
894                 Machine Learning: AI Engineer
895                     Machine Learning Engineer
896             Applied Machine Learning Engineer
897                     Machine Learning Engineer
Name: job_title, Length: 898, dtype: object

In [8]:
pd.DataFrame(df.loc[0]).T

,job_title,company,location,is_remote,salary,post_date,date_accessed,job_description
0,Data Entry,TransPerfect Translations Inc.,Remote,1,,6 days ago,2021-03-05,We are looking for Data Entry Clerks to join o...


In [9]:
### Regex

In [10]:
df

,job_title,company,location,is_remote,salary,post_date,date_accessed,job_description
0,Data Entry,TransPerfect Translations Inc.,Remote,1,,6 days ago,2021-03-05,We are looking for Data Entry Clerks to join o...
1,Data Scientist,ForMotiv,Remote,1,"$75,000 - $120,000 a year",30+ days ago,2021-03-05,Has it ever occurred to you that as the Intern...
2,Data Scientist,Redzara.com,Remote,1,$35 - $80 an hour,10 days ago,2021-03-05,Only GC / EAD only. No C2CBackground screening...
3,Data Scientist,Nova Collective,Remote,1,$35 - $48 an hour,24 days ago,2021-03-05,Are you a data scientist who is really excited...
4,Early Career Data Scientist - Applied Math,Pacific Northwest National Laboratory,"Seattle, WA",0,,1 day ago,2021-03-05,Organization and Job ID Job ID: 311747 Directo...
...,...,...,...,...,...,...,...,...
893,"Machine Learning Engineer, Ads Relevance",Pinterest,"Palo Alto, CA",0,,24 days ago,2021-03-05,About Pinterest: Millions of people across th...
894,Machine Learning: AI Engineer,"ALTRON, Inc.","Fairfax, VA",0,,30+ days ago,2021-03-05,Altron Inc. is seeking a Machine Learning (ML)...
895,Machine Learning Engineer,Global Payments,"Alpharetta, GA 30022",0,,24 days ago,2021-03-05,"Every day, Global Payments makes it possible f..."
896,Applied Machine Learning Engineer,Anomaly,"New York, NY 10012",0,,30+ days ago,2021-03-05,About Anomaly Anomaly is a healthcare technol...


## [FAIL] Finding each Label by Indexing
### The data is sort of in order by the keyword I used while scraping

- 1st: 'data scientist'
- 2nd: 'data analyst'
- 3rd: 'data engineer'
- 4th: 'machine learning engineer'

*I scraped about the same amount of jobs for each of the 4 labels. So I'm going to index the dataframe for the range within each quartile because I know that the job titles will change around that point so I'll just differentiate the labels by peeking into the DataFrame from the reference points.*

In [11]:
print(f'1st quarter: {898*.25} \n 2nd quarter: {898*.5} \n 3rd quarter: {898*.75}')

1st quarter: 224.5 
 2nd quarter: 449.0 
 3rd quarter: 673.5


In [12]:
# the last data scientist entry was at 223
len(df[:224])

224

In [13]:
# the last data analyst entry was at 447
len(df[224:447])

223

In [14]:
# the last data engineer entry was at 676
len(df[447:674])

227

In [15]:
# the rest of the jobs are machine learning engineers
len(df[674:])

224

In [16]:
ds = ['DS' for n in range(0,224)]
da = ['DA' for n in range(0,223)]
de = ['DE' for n in range(0,227)]
mle = ['MLE' for n in range(0,224)]

labels = ds + da + de + mle
len(labels)

898

In [17]:
# Add labels column from the labels list
df['label'] = labels

In [18]:
#peek at sample to see if code is effective
df[['job_title', 'label']].sample(5)

,job_title,label
67,"Data Scientist, Buyer Experience",DS
52,Data Scientist,DS
408,Data Analyst Strategy,DA
871,Sr. Machine Learning Engineer - Discovery,MLE
756,Automation/Artificial Intelligence Machine Lea...,MLE


In [19]:
import re
# checking to be sure any stray jobs that aren't a target be removed
df['valid'] = df.job_title.str.lower().str.extract('(.*?data.*?scientist.*?|.*?data.*?analyst.*?|data.*?engineer.*?|.*?machine.*?learning.*?engineer.*?)')

In [20]:
# 58 nulls
null_rows = df[df.valid.isnull()]
null_rows

,job_title,company,location,is_remote,salary,post_date,date_accessed,job_description,label,valid
0,Data Entry,TransPerfect Translations Inc.,Remote,1,,6 days ago,2021-03-05,We are looking for Data Entry Clerks to join o...,DS,NaN
23,"Associate Scientist, Data Support",Pfizer,"Groton, CT",0,,2 days ago,2021-03-05,"Pfizer colleagues have many perspectives, many...",DS,NaN
33,IT Data Analytics Intern,Farmers Insurance Group,Remote,1,$20.50 an hour,24 days ago,2021-03-05,IT Intern: IT Data Analytics intern We’re loo...,DS,NaN
194,Deep Learning Scientist,SimBioSys,"Champaign, IL",0,,11 days ago,2021-03-05,SimBioSys (http://www.simbiosys.com) is seekin...,DS,NaN
246,"Analyst, Data and Analysis",Digitas,"New York, NY 10104",0,,1 day ago,2021-03-05,"What you’ll do: As an Analyst, you’ll lay the ...",DA,NaN
260,"Analyst, Data and Analysis",Digitas,"New York, NY 10104",0,,1 day ago,2021-03-05,"What you’ll do: As an Analyst, you’ll lay the ...",DA,NaN
264,Data Specialist,Roman Catholic Bishop of Salt Lake City,"Salt Lake City, UT 84103",0,"$39,000 - $45,000 a year",1 day ago,2021-03-05,DATA SPECIALISTOFFICE OF STEWARDSHIP & DEVELOP...,DA,NaN
318,"Analyst, Data & Research",Los Angeles Homeless Services Authority,"Los Angeles, CA 90017",0,"$52,854 - $76,638 a year",1 day ago,2021-03-05,The Los Angeles Homeless Services Authority (L...,DA,NaN
343,Data Specialist,Wunderman Thompson,"Louisville, CO",0,,1 day ago,2021-03-05,Who we are: At Wunderman Thompson we exist to...,DA,NaN
348,Security Analyst,Abnormal Security,"Lehi, UT",0,"$30,000 a year",30+ days ago,2021-03-05,About Abnormal Security: Abnormal Security is...,DA,NaN


### Clean up the rest of the Labels (58)

The data scientist entries dont seem to be true data scientist but data entry or research scientist so I'll drop those (4)


In [21]:
# delete
ds_drop = null_rows[0:4]

In [22]:
ds_drop

,job_title,company,location,is_remote,salary,post_date,date_accessed,job_description,label,valid
0,Data Entry,TransPerfect Translations Inc.,Remote,1,,6 days ago,2021-03-05,We are looking for Data Entry Clerks to join o...,DS,NaN
23,"Associate Scientist, Data Support",Pfizer,"Groton, CT",0,,2 days ago,2021-03-05,"Pfizer colleagues have many perspectives, many...",DS,NaN
33,IT Data Analytics Intern,Farmers Insurance Group,Remote,1,$20.50 an hour,24 days ago,2021-03-05,IT Intern: IT Data Analytics intern We’re loo...,DS,NaN
194,Deep Learning Scientist,SimBioSys,"Champaign, IL",0,,11 days ago,2021-03-05,SimBioSys (http://www.simbiosys.com) is seekin...,DS,NaN


Remove the engineer positions that don't correspond to the work of a data engineer (2)

In [23]:
# delete
de_drop = null_rows[null_rows.label == "DE"]

In [24]:
de_drop

,job_title,company,location,is_remote,salary,post_date,date_accessed,job_description,label,valid
651,Sr Network Engineer-Data Center,Tesla,"Austin, TX",0,,21 days ago,2021-03-05,Role: Tesla is currently seeking a Senior Engi...,DE,NaN
667,Project Engineer – Legacy Sustainment Data,LOCKHEED MARTIN CORPORATION,"Orlando, FL 32825",1,,3 days ago,2021-03-05,This position is a member of the Lockheed Mart...,DE,NaN


Going to remove titles that don't have 'analyst included, like data specialist which is essentially a different position (regex for analyst)

In [25]:
# Keep
mask = []

for row in null_rows.job_title:
    if re.search(r'analyst', row, flags=re.IGNORECASE) != None:
        mask.append(True)
    else:
        mask.append(False)

In [26]:
da_keep = null_rows[mask]

As with the data analyst, most position are in fact mle's but I'll use regex to keep 'learning engineer' and 'machine learning' to filter out the rest.

In [27]:
# keep 
mask = []

for row in null_rows.job_title:
    if re.search(r'machine learning|learning engineer', row, flags=re.IGNORECASE) != None:
        mask.append(True)
    else:
        mask.append(False)

In [28]:
mle_keep = null_rows[mask]

### Delete rows

In [29]:
rows_to_keep = null_rows.drop(ds_drop.index).drop(de_drop.index)
print(len(rows_to_keep))

52


In [30]:
# drop original 58 that needed to be validated
df = df.drop(null_rows.index)

In [31]:
# merge the rows that we decided to keep and drop the valid column
df = pd.concat([df, rows_to_keep]).drop(columns='valid')

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 892 entries, 1 to 892
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job_title        892 non-null    object
 1   company          892 non-null    object
 2   location         892 non-null    object
 3   is_remote        892 non-null    int64 
 4   salary           892 non-null    object
 5   post_date        892 non-null    object
 6   date_accessed    892 non-null    object
 7   job_description  892 non-null    object
 8   label            892 non-null    object
dtypes: int64(1), object(8)
memory usage: 69.7+ KB


In [33]:
df.sample(10)

,job_title,company,location,is_remote,salary,post_date,date_accessed,job_description,label
292,Clinical Data Analyst,Huron Valley Physicians Association,"Ann Arbor, MI 48105",0,,1 day ago,2021-03-05,Clinical Data Analyst Position Summary: The Da...,DA
557,Data Engineer,"Amick Brown, LLC","Sunnyvale, CA 94086",0,,3 days ago,2021-03-05,Amick Brown is seeking an experienced Data Eng...,DE
357,Data Analyst (entry level),Lumen Bioscience,"Seattle, WA 98103",1,From $18 an hour,20 days ago,2021-03-05,Position Summary: Lumen is seeking a highly mo...,DA
894,Machine Learning: AI Engineer,"ALTRON, Inc.","Fairfax, VA",0,,30+ days ago,2021-03-05,Altron Inc. is seeking a Machine Learning (ML)...,MLE
750,Machine Learning Engineer,DeepHealth,"Cambridge, MA",0,,30+ days ago,2021-03-05,Description DeepHealth is a digital health sta...,MLE
545,Data Engineer,"Alteryx, Inc.","Ann Arbor, MI",1,,2 days ago,2021-03-05,"We’re looking for problem solvers, innovators,...",DE
459,Data Engineer,Akshaya Inc,"San Jose, CA",0,$70 - $80 an hour,2 days ago,2021-03-05,DutiesDevelop image processing algorithms usin...,DE
441,Data Analyst (Entry Level),National Institute for Safety Research,"York, PA 17402",1,"From $34,840 a year",23 days ago,2021-03-05,"National Institute for Safety Research, Inc. J...",DA
469,Data Engineer,ProFocus,United States,1,$45 - $60 an hour,2 days ago,2021-03-05,TITLE: Data Engineer with SQL Server Integrati...,DE
590,Data Engineer Remote,Pinnacle Alliances,"Baltimore, MD",1,$65 - $70 an hour,16 days ago,2021-03-05,DATA ENGINEERRemoteUSC/GC§ Data engineers buil...,DE


### Looks better!

In [39]:
from prepare import prep_create_labels
prep_df = prep_create_labels(pd.read_json('indeed-data-jobs-FINAL.json'))

In [40]:
(prep_df == df).sum()

job_title          892
company            892
location           892
is_remote          892
salary             892
post_date          892
date_accessed      892
job_description    892
label              892
dtype: int64